# Illinois Dashboard - Day 3

In this notebook, you will build on your work from Day 2 by creating a simple dashboard that runs the queries from the previous day's notebook. To accomplish this, you will do the following:

- Import UI Components (dropdowns, sliders, and buttons) that will be used for setting parameters of the SQL query.
- Rewrite query as a template query so that it can be built from our inputs
- Place query inside of a function so that it can be run at the click of a button

## Python Setup

In [ ]:
# Package for database connection
from sqlalchemy import create_engine

# Packages for data manipulation
import pandas as pd
import numpy as np
import geopandas as gpd

# Packages for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

# Ignore warnings. This is to prevent distracting notices of new packages that are unnecessary
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Database connection
engine = create_engine('postgresql://@10.10.2.10/appliedda')

## Import User Interaction (UI) Components

Since building User Interface components is outside the scope of this course, these have been definied in an external file named `ui.py` in a class named `DashUI`. Importing this class allows us to use these components.

In [ ]:
from ui import DashUI

# Define metrics to plot
# list_of_metrics dictionary - Key displays in dropdown. Value is injected into SQL query
list_of_metrics = {'Jobs': 'jobs', 'Average Quarterly Earnings': 'avg_wage'}
# Creates dashboard
dash = DashUI(list_of_metrics)

In [ ]:
# Displays dashboard
display(dash.input_panel)

## Write Query Templates

Instead of hardcoding values for the quarters and years of interest as we did in Day 2, we can create a template of our query and fill in these values dynamically based on the values we have selected in our UI components.

This is done by placing a variable name inside of curly braces, like this: **`{var}`**, to mark where a substition should be made. Then, the **`format`** function will replace the variable with the value that we assign to it in the function. Below this is done using **`{y}`** and **`{q}`** in our query string. We then use **`qry.format()`** to replace them with the values from the time slider in the dashboard.

Try adjusting the time slider in the UI above and then re-running the cells below and notice how the parameters of the query change in the printed output.

### Count of Jobs and Average Earnings Query

In [ ]:
count_qry = """
select cnty, count(*) as jobs, avg(wage) as avg_wage
from ada_18_uchi.dashboard_data_il_jobs_rs
where year = {y} and qtr = {q}
group by cnty
order by cnty
"""

In [ ]:
display(dash.input_panel)

In [ ]:
qry = count_qry.format(q=dash.time_slider.value[0], y=dash.time_slider.value[1])
print(qry)

### Change in Jobs and Average Earnings Query

We can do the same thing with the query that calculates the change in jobs and average earnings over time. Click the `Change` button, then use the `Time Range` slider to select a start and end quarter.

In [ ]:
change_qry = '''
select a.cnty, 
    cast(b.jobs - a.jobs as decimal)/a.jobs as change_in_jobs_pct, 
    cast(b.avg_wage - a.avg_wage as decimal)/a.avg_wage as change_in_avg_wage_pct
from(
    select cnty, 
        count(*) as jobs, 
        avg(wage) as avg_wage
    from ada_18_uchi.dashboard_data_il_jobs_rs
    where year = {y0} and qtr = {q0} 
    group by cnty
) as a
full join (
    select cnty, 
        count(*) as jobs, 
        avg(wage) as avg_wage
    from ada_18_uchi.dashboard_data_il_jobs_rs
    where year = {y1} and qtr = {q1}
    group by cnty
) as b
on a.cnty = b.cnty
order by cnty
'''

In [ ]:
display(dash.input_panel)

In [ ]:
qry = change_qry.format(q0=dash.time_range_slider.value[0][0],
                 y0=dash.time_range_slider.value[0][1],
                 q1=dash.time_range_slider.value[1][0],
                 y1=dash.time_range_slider.value[1][1])
print(qry)

## Get County Shapes with `geopandas`

As in the previous notebook, we need to pull in the shape data of the counties in Illinois. These will be stored in a GeoDataFrame names `counties` that will be used in our plotting function.

In [ ]:
### statefp: 17 for IL ###
qry = """
SELECT countyfp, name,
    ST_Transform(geom, 102698) geom 
FROM tl_2016_us_county 
WHERE statefp = '17'
"""
counties = gpd.read_postgis(qry, engine, geom_col='geom')
counties['coords'] = counties.geometry.apply(lambda x: x.representative_point().coords[0])

## Define functions to query and plot the data

Here we define two functions:
1. `run_query`: This runs the query templates that were defined above and uses the selections from our dashboard as parameters in the query.
2. `generate_plot`: This calls the `run_query` function, merges the data with our county data, and plots the map.

Lastly, we need to link the `generate_plot` function to the button in our dashboard, which is done in the last cell.

In [ ]:
# Define run_query function which will run the query templates defined above
def run_query():
    # Get toggle value to determine which plot type to use
    plot_type = dash.plot_toggle.value
    
    # Set query for either Count or Change
    if plot_type == 'Count':
        qry = count_qry.format(q=dash.time_slider.value[0],
                               y=dash.time_slider.value[1])
    elif plot_type == 'Change':
        qry = change_qry.format(q0=dash.time_range_slider.value[0][0],
                                y0=dash.time_range_slider.value[0][1],
                                q1=dash.time_range_slider.value[1][0],
                                y1=dash.time_range_slider.value[1][1])

    # Run the query and return the Dataframe
    df = pd.read_sql(qry, engine)
    return df

In [ ]:
def generate_plot(button_obj):
    dash.output.clear_output()
    with dash.output:
        # Configure plot settings
        sns.set_style('white')
        f, ax = plt.subplots(1, figsize=(6,8))
        ax.axis('off')
        
        # Plot basemap so counties with no data appear hatched
        counties.plot(ax=ax, edgecolor='black', color='lightgray', hatch='//')
        
        # Query and merge data with county shapefile
        df = run_query()
        cnty_df = pd.merge(counties, df, left_on=['countyfp'], right_on=['cnty'])
        
        # If no counties data, show empty map and return
        if len(cnty_df) == 0:
            plt.show()
            return
        
        # Plot county data
        metric = dash.metric_dropdown.value
        if dash.plot_toggle.value == 'Count':
            colmap = sns.cubehelix_palette(8, start=2.9, rot=0, dark=.1, light=.95, as_cmap=True)
            cnty_df.plot(metric, ax=ax, legend=True, edgecolor='black', cmap=colmap)
        elif dash.plot_toggle.value == 'Change':
            metric = 'change_in_{}_pct'.format(metric)
            bound = cnty_df[metric].abs().max()
            colmap = sns.diverging_palette(10, 150, center='light', as_cmap=True)
            cnty_df.plot(metric, ax=ax, legend=True, edgecolor='black', cmap=colmap, vmax=bound, vmin=bound*-1)
        
        # Show plot
        plt.show()

In [ ]:
# Link generate_plot function to Generate Plot button
dash.generate_button.on_click(generate_plot)

In [ ]:
# Display the input panel and the output of the dashboard
display(dash.input_panel)
display(dash.output)